In [1]:
import pickle
import shutil
from enum import StrEnum
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
import sklearn
import xgboost as xgb
from ipywidgets import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.inspection import PartialDependenceDisplay

plt.rcParams["figure.figsize"] = [19, 6]
plt.rcParams["figure.dpi"] = 100

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
class Models(StrEnum):
    SVM = "SVM"
    XGB = "XGB"
    RF = "RF"
    GB = "GB"

# Train

## Data Preparation

In [3]:
short_to_long = {
    "T_AVG": "TEMPERATURE",
    "DO_AVG": "DISSOLVED_OXYGEN",
    "PH_AVG": "PH",
    "COND_AVG": "CONDUCTIVITY",
    "BOD_AVG": "BIO_OXYGEN_DEMAND",
    "N_AVG": "NITRATE_CONC",
    "FC_AVG": "FECAL_COLIFORM",
    "TC_AVG": "TOTAL_COLIFORM",
}

long_to_short = {
    "TEMPERATURE": "T_AVG",
    "DISSOLVED_OXYGEN": "DO_AVG",
    "PH": "PH_AVG",
    "CONDUCTIVITY": "COND_AVG",
    "BIO_OXYGEN_DEMAND": "BOD_AVG",
    "NITRATE_CONC": "N_AVG",
    "FECAL_COLIFORM": "FC_AVG",
    "TOTAL_COLIFORM": "TC_AVG",
}

In [4]:
# (River, State) -> x-axis: Features, y-axis: RMSE value for prediction of features.

df: pd.core.frame.DataFrame = (
    pd.read_csv(f"./WQ 2012_2021.csv")
    .dropna()
    .query("YEAR != 2021")
    .drop(columns=["CODE", "LOCATION", "YEAR"])
    # .bfill()
)

df = pd.concat([df.iloc[:, :2], df.iloc[:, 4::3]], axis=1)
feature_target = df.columns[2:]
df[feature_target] = MinMaxScaler().fit_transform(df.iloc[:, 2:])


def get_train_data(target_name: str, grouped: bool = False):
    def f(x):
        x.rename(
            columns={key: short_to_long[key] for key in x.columns.to_list()[2:]},
            inplace=True,
        )
        return (
            x.groupby(["RIVER", "STATE"]).mean()
            if grouped
            else x.drop(columns=["RIVER", "STATE"])
        )

    return tuple(
        map(
            f,
            train_test_split(
                df.drop(columns=[long_to_short[target_name]]),
                df[["RIVER", "STATE", long_to_short[target_name]]],
                random_state=0,
                test_size=0.1,
            ),
        )
    )

## Training

In [5]:
def train_model(model_type, params):
    fn_params = {
        k: v
        for k, v in params.items()
        if k not in ["X_train", "y_train", "X_valid", "y_valid"]
    }
    match model_type:
        case Models.SVM:
            reg = sklearn.svm.SVR(probabitity=True)
            reg.fit(params["X_train"], params["y_train"].to_numpy().ravel())

        case Models.XGB:
            reg = xgb.XGBRegressor(
                **fn_params,
            )
            reg.fit(
                params["X_train"],
                params["y_train"],
                eval_set=[(params["X_valid"], params["y_valid"])],
            )

        case Models.GB:
            reg = sklearn.ensemble.GradientBoostingRegressor(**fn_params)
            reg.fit(params["X_train"], params["y_train"].to_numpy().ravel())

        case Models.RF:
            reg = sklearn.ensemble.RandomForestRegressor(**fn_params)
            reg.fit(params["X_train"], params["y_train"].to_numpy().ravel())

        case _:
            raise NotImplementedError(f"`{model_type}` model not Implemented!")

    return reg

In [6]:
def target_train(model_type, target_name):
    X_train, X_valid, y_train, y_valid = get_data(target_name)

    params = {
        Models.SVM: {"X_train": X_train, "y_train": y_train},
        Models.XGB: {
            "n_estimators": 10000,
            "early_stopping_rounds": 50,
            "random_state": 0,
            "learning_rate": 0.01,
            "base_score": np.mean(X_train),
            "subsample": 0.5,
            "eval_metric": "squared_error",
            "X_train": X_train,
            "y_train": y_train,
            "X_valid": X_valid,
            "y_valid": y_valid,
        },
        Models.RF: {
            "n_estimators": 1000,
            "max_depth": 10,
            "random_state": 0,
            "max_samples": 0.5,
            "criterion": "squared_error",
            "X_train": X_train,
            "y_train": y_train,
        },
        # GridSearchCV verified
        Models.GB: {
            "n_estimators": 1000,
            "learning_rate": 0.01,
            "max_depth": 10,
            "random_state": 0,
            "subsample": 0.5,
            "loss": "squared_error",
            "X_train": X_train,
            "y_train": y_train,
        },
    }

    return train_model(model_type, params[model_type])

In [7]:
def target_wise_train_model(model_type, target_names, cache=True):
    target_wise_models = {}

    if not cache:
        shutil.rmtree(f"./model/{model_type}/", ignore_errors=True)

    if not Path(f"./model/{model_type}/").exists():
        os.mkdir(f"./model/{model_type}/")

    write_to_disk = False

    if cache and len(paths := list(Path().glob(f"model/{model_type}/*.pkl"))) == len(
        long_to_short.keys()
    ):
        for path in paths:
            with open(path, "rb") as file:
                target_wise_models[path.name.split(".")[0]] = pickle.load(file)
        return target_wise_models
    else:
        write_to_disk = True

    # Display Progress #
    prog = widgets.IntProgress(
        value=0, min=0, max=len(list(long_to_short.keys())) - 1, step=1
    )
    prog_text = widgets.HTML(
        value=f"Training model for <b>{list(long_to_short.keys())[prog.value-1]}</b>"
    )
    display(widgets.VBox([prog, prog_text]))
    #! Display Progress !#

    for target, target_name in enumerate(target_names):
        prog.value = target
        prog_text.value = (
            f"Training model for <b>{list(long_to_short.keys())[target]}</b>".replace(
                "_", " "
            )
        )

        target_wise_models[target_name] = target_train(model_type, target_name)

        if write_to_disk:
            with open(f"./model/{model_type}/{target_name}.pkl", "wb") as file:
                pickle.dump(
                    target_wise_models[target_name],
                    file,
                    protocol=pickle.HIGHEST_PROTOCOL,
                )

    return target_wise_models

# Inference

In [8]:
df_test_1: pd.core.frame.DataFrame = (
    pd.read_csv(f"./WQ 2012_2021.csv")
    .dropna()
    .query("YEAR == 2021")
    .drop(columns=["LOCATION", "YEAR"])
    .bfill()
    .groupby(["RIVER", "STATE", "CODE"])
    .mean()
)
river_state_station = df_test_1.index.to_list()
index = df_test_1.index
df_test_1 = df_test_1.iloc[:, 2::3]
df_test_1.rename(
    columns={key: short_to_long[key] for key in df_test_1.columns.to_list()},
    inplace=True,
)
df_test_1[list(long_to_short.keys())] = MinMaxScaler().fit_transform(df_test_1)

df_test: pd.core.frame.DataFrame = (
    pd.read_csv(f"./WQ 2012_2021.csv")
    .dropna()
    .query("YEAR == 2021")
    .drop(columns=["CODE", "LOCATION", "YEAR"])
    .bfill()
    .groupby(["RIVER", "STATE"])
    .mean()
)
river_state = df_test.index.to_list()
index = df_test.index
df_test = df_test.iloc[:, 2::3]
df_test.rename(
    columns={key: short_to_long[key] for key in df_test.columns.to_list()},
    inplace=True,
)
feature_target = df_test.columns
df_test[feature_target] = MinMaxScaler().fit_transform(df_test)

dict_river_state_station = {}
for r, s, c in river_state_station:
    if r in dict_river_state_station:
        if s in dict_river_state_station[r]:
            dict_river_state_station[r][s].append(c)
            continue
        dict_river_state_station[r][s] = [c]
        continue
    dict_river_state_station[r] = {s: [c]}

dict_river_state = {}
for r, s, _ in river_state_station:
    if r in dict_river_state:
        dict_river_state[r].add(s)
        continue
    dict_river_state[r] = {s}

In [9]:
target_d_1 = widgets.Dropdown(options=[], value=None, description="Target: ")
river_d_1 = widgets.Dropdown(
    options=list(dict_river_state.keys()),
    value=None,
    description="River:",
)
state_d_1 = widgets.Dropdown(
    description="State:",
)
station_d_1 = widgets.Dropdown(
    description="Station:",
)


target_d = widgets.Dropdown(options=[], value=None, description="Target: ")
model_d = widgets.Dropdown(options=Models, value=None, description="Model: ")
river_d = widgets.Dropdown(
    options=list(dict_river_state.keys()),
    value=None,
    description="River:",
)
state_d = widgets.Dropdown(
    description="State:",
)

In [10]:
def calculate_error_1(targets_model, river, state, station):
    err = []
    trgt = []
    for target_name in long_to_short.keys():
        model = targets_model[target_name]

        X = (
            df_test_1.drop(columns=[target_name])
            .loc[river, state, station]
            .to_frame()
            .transpose()
        )
        y = df_test_1[target_name].loc[river, state, station]

        o = model.predict(X)
        rse = np.linalg.norm(o - y)
        err.append(rse)
        trgt.append(target_name)
    return trgt, err


def calculate_error(targets_model, river, state):
    err = []
    trgt = []
    for target_name in long_to_short.keys():
        model = targets_model[target_name]

        X = df_test.drop(columns=[target_name]).loc[river, state].to_frame().transpose()
        y = df_test[target_name].loc[river, state]

        o = model.predict(X)
        rse = np.linalg.norm(o - y)
        err.append(rse)
        trgt.append(target_name)
    return trgt, err

### Error Plot

In [11]:
def plot_error_1(targets_model, river, state, station):
    if river == None or state == None:
        return None

    plt.close()

    X, y = calculate_error_1(targets_model, river, state, station)
    plt.bar(X, y)
    plt.scatter(X, y)
    plt.title(f"{river}\n{state}: {station}")
    plt.xlabel("Target perdicted")
    plt.ylabel("RMSE")

    plt.show()


def plot_error(targets_model, river, state):
    if river == None or state == None:
        return None

    plt.close()

    X, y = calculate_error(targets_model, river, state)
    plt.bar(X, y)
    plt.scatter(X, y)
    plt.title(f"{river}\n{state}")

    plt.show()

### Explainer PLot

In [16]:
def explainer_plot(target_name, targets_model):
    if not (path:=Path(f"images/{model_d.value}/shap")).exists():
        path.mkdir(parents=True, exist_ok=True)
    if not (path:=Path(f"images/{model_d.value}/pdp/{target_name}")).exists():
        path.mkdir(parents=True, exist_ok=True)
    
    if target_name == None or targets_model == None:
        return None

    plt.close()
    model = targets_model[target_name]
    X_train, _, y_train, _ = get_train_data(target_name)

    explainer = shap.Explainer(lambda x: model.predict(x), X_train)

    if not Path(f"./shap_values/{model_d.value}").exists():
        os.mkdir(f"./shap_values/{model_d.value}")

    path = Path(f"./shap_values/{model_d.value}/{target_name}.pkl")
    if not path.exists():
        shap_values = explainer(X_train)
        with open(path, "wb") as file:
            pickle.dump(shap_values, file, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(path, "rb") as file:
            shap_values = pickle.load(file)

    plt.title(f"{target_name}\n{model_d.value}")

    shap.summary_plot(
        shap_values,
        X_train,
        alpha=0.5,
        title=target_name,
        show=False
    )
    plt.savefig(f"images/{model_d.value}/shap/summary_{target_name}")
    plt.show()
    
    shap.plots.bar(shap_values, show=False)
    plt.savefig(f"images/{model_d.value}/shap/bar_{target_name}")
    plt.show()

    for i in range(7):
        PartialDependenceDisplay.from_estimator(model, X_train, [i], n_cols=2)
        plt.title(f"{target_name}")
        plt.savefig(f"images/{model_d.value}/pdp/{target_name}/{''.join(X_train.columns[i])}")
    
    # PartialDependenceDisplay.from_estimator(model, X_train, [1, 2], n_cols=2)
    # plt.savefig(f"images/pdp/{target_name}/{''.join(X_train.columns[1:3])}")
    
    # PartialDependenceDisplay.from_estimator(model, X_train, [2, 3], n_cols=2)
    # plt.savefig(f"images/pdp/{target_name}/{''.join(X_train.columns[2:4])}")
    
    # PartialDependenceDisplay.from_estimator(model, X_train, [3, 4], n_cols=2)
    # plt.savefig(f"images/pdp/{target_name}/{''.join(X_train.columns[3:5])}")
    
    # PartialDependenceDisplay.from_estimator(model, X_train, [5, 6], n_cols=2)
    # plt.savefig(f"images/pdp/{target_name}/{''.join(X_train.columns[4:6])}")
    # shap.partial_dependence_plot(target_name, targets_model[target_name], X_train, model_expected_value=True, feature_expected_value=True, show=False, ice=False)

In [13]:
targets_model = {}


def update_model(change):
    global targets_model
    targets = list(long_to_short.keys())
    targets_model = target_wise_train_model(
        model_type=model_d.value, target_names=targets
    )
    target_d_1.options = targets
    target_d_1.value = targets[0]


def update_state_1(change):
    state_d_1.options = list(dict_river_state_station[river_d_1.value].keys())
    state_d_1.value = list(dict_river_state_station[river_d_1.value].keys())[0]


def update_station_1(change):
    station_d_1.options = dict_river_state_station[river_d_1.value][state_d_1.value]
    station_d_1.value = dict_river_state_station[river_d_1.value][state_d_1.value][0]


def update_state(change):
    state_d.options = dict_river_state[river_d.value]
    state_d.value = list(dict_river_state[river_d.value])[0]


model_d.observe(update_model, "value")
river_d.observe(update_state, "value")
river_d_1.observe(update_state_1, "value")
state_d_1.observe(update_station_1, "value")

display(model_d)

graph_e_1 = widgets.interactive_output(
    lambda target_name: explainer_plot(
        target_name=target_name, targets_model=targets_model
    ),
    {"target_name": target_d_1},
)


graph_p = widgets.interactive_output(
    lambda river, state: plot_error(
        river=river, state=state, targets_model=targets_model
    ),
    {"river": river_d, "state": state_d},
)

graph_p_1 = widgets.interactive_output(
    lambda river, state, station: plot_error_1(
        river=river, state=state, station=station, targets_model=targets_model
    ),
    {"river": river_d_1, "state": state_d_1, "station": station_d_1},
)


explain = widgets.VBox([target_d_1, graph_e_1])
predict_station = widgets.VBox([river_d_1, state_d_1, station_d_1, graph_p_1])
predict_state = widgets.VBox([river_d, state_d, graph_p])

tab = widgets.Tab()
tab.children = [explain, predict_state, predict_station]
tab.titles = ["Explanation", "Prediction (State)", "Prediction (Station)"]
tab

Dropdown(description='Model: ', options=(<Models.SVM: 'SVM'>, <Models.XGB: 'XGB'>, <Models.RF: 'RF'>, <Models.…

## Todo

1. Related work.
2. Compare your work with related work.
3. Grpahs and Tables.
4. Inference derived from Graphs and Tables.

Compare models, by their total rmse.

### Papers not accesible

1. https://www.nature.com/articles/s41598-022-20604-x
2. https://www.sciencedirect.com/science/article/abs/pii/S0341816223000887
3. https://link.springer.com/article/10.1007/s11356-023-27481-5
4. https://link.springer.com/chapter/10.1007/978-981-19-7528-8_19